<a href="https://colab.research.google.com/github/YaraMansour1985/prediction-of-Product-Sales/blob/main/project1part6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/AXSOSACADEMY/01-Fundamentals/Week02/Data/sales_predictions_2023.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
df.info()
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


,0
Item_Identifier,object
Item_Weight,float64
Item_Fat_Content,object
Item_Visibility,float64
Item_Type,object
Item_MRP,float64
Outlet_Identifier,object
Outlet_Establishment_Year,int64
Outlet_Size,object
Outlet_Location_Type,object


In [4]:
df=pd.read_csv('/content/drive/MyDrive/AXSOSACADEMY/01-Fundamentals/Week02/Data/sales_predictions_2023.csv')
df.duplicated().sum()
df=df.drop_duplicates()


In [5]:
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('LF','Low Fat')
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('low fat','Low Fat')
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('reg','Regular')

In [6]:
df['Item_Fat_Content'].value_counts()

,count
Item_Fat_Content,
Low Fat,5517
Regular,3006


In [7]:
#drop colum item identifier
df=df.drop(columns=['Item_Identifier'])

In [8]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, ColumnTransformer

from sklearn import set_config
set_config(transform_output='pandas')

In [9]:
# Define features (X) and target (y).
# X should include every feature except: "Item_Outlet_Sales."
# y should be 'Item_Outlet_Sales'

target = 'Item_Outlet_Sales'
X = df.drop(columns=[target , 'Item_Outlet_Sales'])
y = df[target]

In [10]:
# Train test split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [11]:
# Checking object columns
X_train.select_dtypes('object')

,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
4776,Low Fat,Household,OUT018,Medium,Tier 3,Supermarket Type2
7510,Regular,Snack Foods,OUT018,Medium,Tier 3,Supermarket Type2
5828,Regular,Meat,OUT049,Medium,Tier 1,Supermarket Type1
5327,Low Fat,Baking Goods,OUT035,Small,Tier 2,Supermarket Type1
4810,Low Fat,Frozen Foods,OUT045,NaN,Tier 2,Supermarket Type1
...,...,...,...,...,...,...
5734,Regular,Fruits and Vegetables,OUT010,NaN,Tier 3,Grocery Store
5191,Low Fat,Frozen Foods,OUT017,NaN,Tier 2,Supermarket Type1
5390,Low Fat,Health and Hygiene,OUT045,NaN,Tier 2,Supermarket Type1
860,Low Fat,Snack Foods,OUT017,NaN,Tier 2,Supermarket Type1


In [12]:
# Prepare ordinal pipeline
ord_cols = ['Outlet_Size']

# Impute Most common
impute_common = SimpleImputer(strategy='most_frequent')

# Ordinal Encoder
Outlet_Size_order = ['Small','Medium','High']
ord_encoder = OrdinalEncoder(categories=[Outlet_Size_order])

# Scaler
scaler = StandardScaler()

# make a pipeline
ord_pipe = make_pipeline(impute_common, ord_encoder, scaler)
ord_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Small', 'Medium', 'High']])),
                ('standardscaler', StandardScaler())])

In [13]:
# Prepare Categorical pipeline
cat_cols = X_train.select_dtypes('object').drop(columns=ord_cols).columns

impute_missing = SimpleImputer(strategy='constant',fill_value='MISSING')
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_pipe = make_pipeline(impute_missing, ohe_encoder)
ohe_pipe

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='MISSING', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [14]:
# Prepare numeric piepline
num_cols = X_train.select_dtypes('number').columns

mean_imputer = SimpleImputer(strategy="mean")
scaler = StandardScaler()

#Numeric Pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [15]:
# Create the column transformer
preprocessor = ColumnTransformer([
    ('ord_pipe', ord_pipe, ord_cols),
    ('ohe_pipe', ohe_pipe, cat_cols),
    ('numeric_pipe', numeric_pipe, num_cols)
])

In [16]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('ord_pipe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Small',
                                                                              'Medium',
                                                                              'High']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Outlet_Size']),
                                ('ohe_pipe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 (...
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')),
                                ('numeric_pipe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object'))])

In [17]:
# Prepare numeric piepline
num_cols = X_train.select_dtypes('number').columns

mean_imputer = SimpleImputer(strategy="mean")
scaler = StandardScaler()

#Numeric Pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [18]:
# Create the column transformer
preprocessor = ColumnTransformer([
    ('ord_pipe', ord_pipe, ord_cols),
    ('ohe_pipe', ohe_pipe, cat_cols),
    ('numeric_pipe', numeric_pipe, num_cols)
])

In [19]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('ord_pipe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Small',
                                                                              'Medium',
                                                                              'High']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Outlet_Size']),
                                ('ohe_pipe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 (...
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')),
                                ('numeric_pipe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object'))])

In [20]:
# Transform the training and testing data
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

display(X_train_processed.head())
display(X_test_processed.head())

,ord_pipe__Outlet_Size,ohe_pipe__Item_Fat_Content_Low Fat,ohe_pipe__Item_Fat_Content_Regular,ohe_pipe__Item_Type_Baking Goods,ohe_pipe__Item_Type_Breads,ohe_pipe__Item_Type_Breakfast,ohe_pipe__Item_Type_Canned,ohe_pipe__Item_Type_Dairy,ohe_pipe__Item_Type_Frozen Foods,ohe_pipe__Item_Type_Fruits and Vegetables,...,ohe_pipe__Outlet_Location_Type_Tier 2,ohe_pipe__Outlet_Location_Type_Tier 3,ohe_pipe__Outlet_Type_Grocery Store,ohe_pipe__Outlet_Type_Supermarket Type1,ohe_pipe__Outlet_Type_Supermarket Type2,ohe_pipe__Outlet_Type_Supermarket Type3,numeric_pipe__Item_Weight,numeric_pipe__Item_Visibility,numeric_pipe__Item_MRP,numeric_pipe__Outlet_Establishment_Year
4776,0.287374,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.817249,-0.712775,1.828109,1.327849
7510,0.287374,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.556340,-1.291052,0.603369,1.327849
5828,0.287374,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-0.131512,1.813319,0.244541,0.136187
5327,-1.384048,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,-1.169219,-1.004931,-0.952591,0.732018
4810,0.287374,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.528819,-0.965484,-0.336460,0.493686


,ord_pipe__Outlet_Size,ohe_pipe__Item_Fat_Content_Low Fat,ohe_pipe__Item_Fat_Content_Regular,ohe_pipe__Item_Type_Baking Goods,ohe_pipe__Item_Type_Breads,ohe_pipe__Item_Type_Breakfast,ohe_pipe__Item_Type_Canned,ohe_pipe__Item_Type_Dairy,ohe_pipe__Item_Type_Frozen Foods,ohe_pipe__Item_Type_Fruits and Vegetables,...,ohe_pipe__Outlet_Location_Type_Tier 2,ohe_pipe__Outlet_Location_Type_Tier 3,ohe_pipe__Outlet_Type_Grocery Store,ohe_pipe__Outlet_Type_Supermarket Type1,ohe_pipe__Outlet_Type_Supermarket Type2,ohe_pipe__Outlet_Type_Supermarket Type3,numeric_pipe__Item_Weight,numeric_pipe__Item_Visibility,numeric_pipe__Item_MRP,numeric_pipe__Outlet_Establishment_Year
7503,1.958796,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,3.310089e-01,-0.776646,-0.998816,-1.293807
2957,-1.384048,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-1.179892e+00,0.100317,-1.585194,-0.102145
7031,0.287374,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,3.784469e-01,-0.482994,-1.595784,0.136187
1084,0.287374,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,4.213344e-16,-0.415440,0.506592,-1.532139
856,-1.384048,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,-6.426567e-01,-1.047426,0.886725,0.732018


In [21]:
#save  the dataframe as csv
pd.DataFrame(X_train_processed).to_csv('X_train_processed.csv', index=False)
pd.DataFrame(X_test_processed).to_csv('X_test_processed.csv', index=False)

#Part 6
## Modeling
### Linear regression

In [26]:
#build a linear regression model
from sklearn.linear_model import LinearRegression
lin_reg=LinearRegression()
lin_reg
# fit the model on data
lin_reg.fit(X_train_processed,y_train)

LinearRegression()

In [30]:
#use the custom evaluation function to get the metrics of the model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def regression_metrics(y_true, y_pred, label='', verbose = True, output_dict=False):
  # Get metrics
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mse) # Calculate RMSE manually
  r_squared = r2_score(y_true, y_pred)
  if verbose == True:
    # Print Result with Label and Header
    header = "-"*60
    print(header, f"Regression Metrics: {label}", header, sep='\n')
    print(f"- MAE = {mae:,.3f}")
    print(f"- MSE = {mse:,.3f}")
    print(f"- RMSE = {rmse:,.3f}")
    print(f"- R^2 = {r_squared:,.3f}")
  if output_dict == True:
      metrics = {'Label':label, 'MAE':mae,
                 'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
      return metrics

def evaluate_regression(reg, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False):
  # Get predictions for training data
  y_train_pred = reg.predict(X_train)

  # Call the helper function to obtain regression metrics for training data
  results_train = regression_metrics(y_train, y_train_pred, verbose = verbose,
                                     output_dict=output_frame,
                                     label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = reg.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = regression_metrics(y_test, y_test_pred, verbose = verbose,
                                  output_dict=output_frame,
                                    label='Test Data' )

  # Store results in a dataframe if ouput_frame is True
  if output_frame:
    results_df = pd.DataFrame([results_train,results_test])
    # Set the label as the index
    results_df = results_df.set_index('Label')
    # Set index.name to none to get a cleaner looking result
    results_df.index.name=None
    # Return the dataframe
    return results_df.round(3)

In [31]:
# evaluate the model
evaluate_regression(lin_reg,X_train_processed,y_train,X_test_processed,y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 847.129
- MSE = 1,297,558.136
- RMSE = 1,139.104
- R^2 = 0.562

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 804.120
- MSE = 1,194,349.715
- RMSE = 1,092.863
- R^2 = 0.567


In [ ]:
#compare R^2 in both training and test data
#it is very close to each other with 0.005 difference which means its neither overfitted nor underfitted it is moderately fitted

###Random forest

In [34]:
# build a random forest model to predict sales
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor(random_state=42)
random_forest.fit(X_train_processed, y_train)

RandomForestRegressor(random_state=42)

In [36]:
#evaluate the model
evaluate_regression(random_forest ,X_train_processed,y_train,X_test_processed,y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 296.476
- MSE = 182,669.600
- RMSE = 427.399
- R^2 = 0.938

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 767.288
- MSE = 1,220,665.032
- RMSE = 1,104.837
- R^2 = 0.558


In [ ]:
#compare R^2 in both training and test data
#the R^2 in training data is 0.38 larger than R^2 in test data which means that the model is overfitted

In [ ]:
# compare the two models
#the linear regression mpodel is better in prediction tham the random forest model

###tuning random forest model

In [39]:
#use GridSearchCv to tune two hyperparameters in forest model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
Param_grid_rf= {
    'n_estimators': [50,100, 200], 'max_depth': [None, 5, 10]}
#instantiate gridearchcv and fit it on the training data
random_forest_tuned = RandomForestRegressor(random_state=42)
grid_rf= GridSearchCV(random_forest_tuned, param_grid=Param_grid_rf, cv=5, n_jobs=-1, verbose=1)
grid_rf.fit(X_train_processed, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 10],
                         'n_estimators': [50, 100, 200]},
             verbose=1)

In [40]:
#fit the tuned forest on the training data and evaluate the model , set no folds
grid_rf.fit(X_train_processed, y_train)
evaluate_regression(grid_rf, X_train_processed, y_train, X_test_processed, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 755.047
- MSE = 1,152,172.825
- RMSE = 1,073.393
- R^2 = 0.611

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 729.251
- MSE = 1,100,811.483
- RMSE = 1,049.196
- R^2 = 0.601


In [ ]:
#compare R^2 in both training and test data
#the R^2 in training data is 0.01 larger than R^2 in test data which means that the model is much better and fitted after tunning

#Evaluation

#overall
## Random forest model performance is higher than the linear regression model
1- R^2 (fitting)

model 1 (Linear Regression) R^2 for training is 0.562=(56%)
and for test is 0.567=(57%)
this means that this model explains only 56% of the variation in data
model 2 ( tuned forest model) R^2 for training is 0.611=(61%)
and for test is 0.601=(60%)
this means that it explains 60% of the variation in data

2- Errors(MAE)
the random forest model is off from the actual values by 729 units on average while the linear regression predictions are off from the actual values by 847  m this metric (MAE) is commonly used and easy to understand

3- so Random forest model is slightly more balanced overall  and has more stable performance on both training and test data

